# Instant Gratification

## Your first Frontier LLM Project!

Let's build a useful LLM solution - in a matter of minutes.

By the end of this course, you will have built an autonomous Agentic AI solution with 7 agents that collaborate to solve a business problem. All in good time! We will start with something smaller...

Our goal is to code a new kind of Web Browser. Give it a URL, and it will respond with a summary. The Reader's Digest of the internet!!

Before starting, you should have completed the setup for [PC](../SETUP-PC.md) or [Mac](../SETUP-mac.md) and you hopefully launched this jupyter lab from within the project root directory, with your environment activated.

## If you're new to Jupyter Lab

Welcome to the wonderful world of Data Science experimentation! Once you've used Jupyter Lab, you'll wonder how you ever lived without it. Simply click in each "cell" with code in it, such as the cell immediately below this text, and hit Shift+Return to execute that cell. As you wish, you can add a cell with the + button in the toolbar, and print values of variables, or try out variations.  

I've written a notebook called [Guide to Jupyter](Guide%20to%20Jupyter.ipynb) to help you get more familiar with Jupyter Labs, including adding Markdown comments, using `!` to run shell commands, and `tqdm` to show progress.

## If you'd prefer to work in IDEs

If you're more comfortable in IDEs like VSCode or Pycharm, they both work great with these lab notebooks too.  
If you'd prefer to work in VSCode, [here](https://chatgpt.com/share/676f2e19-c228-8012-9911-6ca42f8ed766) are instructions from an AI friend on how to configure it for the course.

## If you'd like to brush up your Python

I've added a notebook called [Intermediate Python](Intermediate%20Python.ipynb) to get you up to speed. But you should give it a miss if you already have a good idea what this code does:    
`yield from {book.get("author") for book in books if book.get("author")}`

## I am here to help

If you have any problems at all, please do reach out.  
I'm available through the platform, or at ed@edwarddonner.com, or at https://www.linkedin.com/in/eddonner/ if you'd like to connect (and I love connecting!)

## More troubleshooting

Please see the [troubleshooting](troubleshooting.ipynb) notebook in this folder to diagnose and fix common problems. At the very end of it is a diagnostics script with some useful debug info.

## If this is old hat!

If you're already comfortable with today's material, please hang in there; you can move swiftly through the first few labs - we will get much more in depth as the weeks progress.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Please read - important note</h2>
            <span style="color:#900;">The way I collaborate with you may be different to other courses you've taken. I prefer not to type code while you watch. Rather, I execute Jupyter Labs, like this, and give you an intuition for what's going on. My suggestion is that you do this with me, either at the same time, or (perhaps better) right afterwards. Add print statements to understand what's going on, and then come up with your own variations. If you have a Github account, use this to showcase your variations. Not only is this essential practice, but it demonstrates your skills to others, including perhaps future clients or employers...</span>
        </td>
    </tr>
</table>
<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business value of these exercises</h2>
            <span style="color:#181;">A final thought. While I've designed these notebooks to be educational, I've also tried to make them enjoyable. We'll do fun things like have LLMs tell jokes and argue with each other. But fundamentally, my goal is to teach skills you can apply in business. I'll explain business implications as we go, and it's worth keeping this in mind: as you build experience with models and techniques, think of ways you could put this into action at work today. Please do contact me if you'd like to discuss more or if you have ideas to bounce off me.</span>
        </td>
    </tr>
</table>

In [1]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

# Connecting to OpenAI

The next cell is where we load in the environment variables in your `.env` file and connect to OpenAI.

## Troubleshooting if you have problems:

Head over to the [troubleshooting](troubleshooting.ipynb) notebook in this folder for step by step code to identify the root cause and fix it!

If you make a change, try restarting the "Kernel" (the python process sitting behind this notebook) by Kernel menu >> Restart Kernel and Clear Outputs of All Cells. Then try this notebook again, starting at the top.

Or, contact me! Message me or email ed@edwarddonner.com and we will get this to work.

Any concerns about API costs? See my notes in the README - costs should be minimal, and you can control it at every point. You can also use Ollama as a free alternative, which we discuss during Day 2.

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [3]:
openai = OpenAI()

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.
# If it STILL doesn't work (horrors!) then please see the Troubleshooting notebook in this folder for full instructions

# Let's make a quick call to a Frontier model to get started, as a preview!

In [4]:
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "Hello, GPT! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello! Welcome! I’m glad you’re here. How can I assist you today?


In [16]:
response.choices[0].message

ChatCompletionMessage(content='Hello! Welcome! I’m glad you’re here. How can I assist you today?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)

## OK onwards with our first project

In [17]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [20]:
# Let's try one out. Change the website and add print statements to follow along.

ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connect
with me for

In [ ]:
lu = Website("https://en.wikipedia.org/wiki/Lu_Laysreng")
print(lu.title)
print(lu.text)

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [22]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [23]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [24]:
print(user_prompt_for(ed))

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.

In [ ]:
print(user_prompt_for(lu))

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]

To give you a preview, the next 2 cells make a rather simple call - we won't stretch the might GPT (yet!)

In [26]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [27]:
# To give you a preview -- calling OpenAI with system and user messages:

response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
print(response.choices[0].message.content)

Well, if we're sticking to basic arithmetic, 2 + 2 is 4. But if you’re looking for a philosophical debate, I can surely help you ponder the meaning of “2” in a digital age!


## And now let's build useful messages for GPT-4o-mini, using a function

In [28]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [29]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a massive, posi

In [30]:
messages_for(lu)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Lu Laysreng - Wikipedia\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nJump to content\nMain menu\nMain menu\nmove to sidebar\nhide\nNavigation\nMain page\nContents\nCurrent events\nRandom article\nAbout Wikipedia\nContact us\nContribute\nHelp\nLearn to edit\nCommunity portal\nRecent changes\nUpload file\nSearch\nSearch\nAppearance\nDonate\nCreate account\nLog in\nPersonal tools\nDonate\nCreate account\nLog in\nPages for logged out editors\nlearn more\nContributions\nTalk\nContents\nmove to sidebar\nhide\n(Top)\n1\nReferences\nToggle the table of contents\nLu Laysreng\n1 language\nTiếng Việt\nEdit links

## Time to bring it together - the API for OpenAI is very simple!

In [31]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [32]:
summarize("https://edwarddonner.com")

'# Summary of Edward Donner\'s Website\n\nEdward Donner\'s website primarily showcases his interest in coding, large language models (LLMs), and artificial intelligence (AI). He is the co-founder and CTO of Nebula.io, a platform that leverages AI to help individuals discover their potential and enhance talent recruitment processes. The site reflects his background in tech, including his previous role as the founder and CEO of the AI startup untapt, which was acquired in 2021.\n\n## Key Features:\n- **Outsmart**: An arena designed for LLMs to compete against each other in strategies involving diplomacy and cunning.\n- **Personal Interests**: Ed expresses personal hobbies such as DJing, amateur electronic music production, and engagement with tech communities like Hacker News.\n\n## Blog Posts and Announcements:\n- **December 21, 2024**: Welcoming a community of data scientists with the title "Welcome, SuperDataScientists!"\n- **November 13, 2024**: A resource guide titled "Mastering AI 

In [33]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [35]:
display_summary("https://en.wikipedia.org/wiki/Lu_Laysreng")

# Summary of Lu Laysreng - Wikipedia

Lu Laysreng (born 1938) is a prominent Cambodian politician, serving as one of Cambodia's Deputy Prime Ministers and the Minister of Rural Development. A member of the Funcinpec party, he was elected to the National Assembly representing Kampong Speu Province in 2003. 

Born in Phnom Penh to Chinese immigrants, Lu spent part of his childhood in Swatow before pursuing higher education in the United States, specifically at California State University, Long Beach, where he also obtained American citizenship. After returning to Cambodia, he played a significant role in enhancing Cambodia's relations with China and was appointed as the information minister following the 2003 elections. 

Fluent in Khmer, Teochew, and English, Lu is known by his Chinese name "杨来盛" and his Khmer name, which is a transliteration.

The article is a stub, indicating that it is a brief entry that may be expanded with more information.

In [36]:
display_summary("https://edwarddonner.com")

# Summary of Edward Donner's Website

Edward Donner's website features his interests and professional background, particularly focusing on coding and experimenting with Large Language Models (LLMs). He is the co-founder and CTO of **Nebula.io**, a company that leverages AI to enhance talent discovery and management, having previously founded the AI startup untapt, which was acquired in 2021. 

## Recent Posts and Announcements
- **December 21, 2024**: Welcoming "SuperDataScientists!"
- **November 13, 2024**: Sharing resources for "Mastering AI and LLM Engineering."
- **October 16, 2024**: Providing resources on transitioning from Software Engineer to AI Data Scientist.
- **August 6, 2024**: Introduction to the "Outsmart LLM Arena," which involves competitive scenarios for LLMs focused on diplomacy and strategy. 

Overall, the website reflects Edward's passion for AI, technology, and his ongoing engagement in the field with insightful resources for fellow enthusiasts.

# Let's try more websites

Note that this will only work on websites that can be scraped using this simplistic approach.

Websites that are rendered with Javascript, like React apps, won't show up. See the community-contributions folder for a Selenium implementation that gets around this. You'll need to read up on installing Selenium (ask ChatGPT!)

Also Websites protected with CloudFront (and similar) may give 403 errors - many thanks Andy J for pointing this out.

But many websites will work just fine!

In [37]:
display_summary("https://cnn.com")

# Summary of CNN Website

CNN is a leading news platform that provides coverage on a wide range of topics, including:

- **US News**: Current events, politics, crime, and justice in the United States.
- **World News**: Global affairs, including major regions like Europe, Asia, Africa, and the Middle East.
- **Politics**: In-depth coverage of governmental issues, elections, and political analysis.
- **Business**: Updates on economy, markets, and major corporate news.
- **Health**: Insights into medical advancements, health tips, and wellness news.
- **Entertainment**: Coverage of movies, television, celebrity news, and cultural events.
- **Science and Climate**: Articles on scientific discoveries, environmental issues, and climate change.
- **Sports**: Updates on various sports disciplines, including NFL, NBA, and international games.

### Recent News Highlights:
- **Ukraine-Russia War**: Ongoing coverage and analysis of the conflict.
- **Israel-Hamas War**: Reports on the situation, including human interest stories like the emotional reunion of hostages with their families.
- **Trump Administration Developments**: News on executive actions, including immigration policies and cabinet confirmations.

CNN also offers video content, podcasts, and interactive segments related to trending stories and significant events, like presidential inaugurations.

Overall, CNN aims to keep its audience informed with up-to-date news and analysis on crucial domestic and international issues.

In [38]:
display_summary("https://anthropic.com")

# Summary of Anthropic Website

Anthropic is an AI safety and research company based in San Francisco, focused on creating reliable and beneficial AI systems. Their latest AI model, **Claude 3.5 Sonnet**, is now available for interaction, along with a new offering called **Claude 3.5 Haiku**, which was announced on October 22, 2024.

## Key Highlights

- **Claude Models**: Anthropic offers various Claude models, including the latest 3.5 Sonnet and 3.5 Haiku, designed for advanced AI interactions.
- **AI Safety Research**: The company emphasizes AI safety and responsible development practices, with ongoing research in the field.
- **Enterprise Solutions**: Anthropic provides AI solutions tailored for enterprise applications.

## Recent Announcements

- **Oct 22, 2024**: Introduction of new models, Claude 3.5 Sonnet and Claude 3.5 Haiku.
- **Sep 4, 2024**: Launch of Claude for Enterprise, aimed at business applications.
- **Research Initiative**: The company published insights on "Constitutional AI" focusing on ensuring harmlessness through AI feedback (Dec 15, 2022).
- A discussion on "Core Views on AI Safety" was released on March 8, 2023, detailing their perspectives on AI development.

Through their products and research, Anthropic prioritizes safety in developing AI technologies.

In [40]:
display_summary("https://ramp.com/")

# Ramp Website Summary

Ramp offers a comprehensive spend management solution designed to streamline finance operations for businesses. The platform includes features such as corporate cards, accounts payable automation, and smart expense management, aimed at reducing manual processes and improving efficiency.

## Key Features:
- **Corporate Cards:** Automatically matches receipts to transactions and learns from spending patterns.
- **Expense Management:** Simplifies the submission and approval process for expenses.
- **Accounts Payable:** Automated bill payment processes to eliminate repetitive tasks.
- **Procurement:** Streamlined intake-to-pay process for faster operations.
- **Travel Management:** Corporately managed travel booking that adheres to established policies.
- **Accounting Automation:** Integrates seamlessly with existing systems for faster book closing.

## Customer Impact:
Ramp has saved over 10 million hours for more than 30,000 customers, with many testimonials citing significant time savings and improved financial oversight.

## Recognition:
Ramp has been recognized in various lists for its innovative approach to spend management, including:
- **Forbes Cloud 100 2024**
- **CNBC Disruptor 50 2024**
- **G2 Best Software List 2024**

## Call to Action:
Interested businesses can sign up for a free demo to explore how Ramp can enhance their financial operations.

For more information, businesses are encouraged to get started for free through their website.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise, you experienced calling the Cloud API of a Frontier Model (a leading model at the frontier of AI) for the first time. We will be using APIs like OpenAI at many stages in the course, in addition to building our own LLMs.

More specifically, we've applied this to Summarization - a classic Gen AI use case to make a summary. This can be applied to any business vertical - summarizing the news, summarizing financial performance, summarizing a resume in a cover letter - the applications are limitless. Consider how you could apply Summarization in your business, and try prototyping a solution.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you continue - now try yourself</h2>
            <span style="color:#900;">Use the cell below to make your own simple commercial example. Stick with the summarization use case for now. Here's an idea: write something that will take the contents of an email, and will suggest an appropriate short subject line for the email. That's the kind of feature that might be built into a commercial email tool.</span>
        </td>
    </tr>
</table>

In [41]:
# Step 1: Create your prompts
system_prompt = "You are an assistant that analyzes emails and creates a subject line that makes sense based on the context of the email. \
                The emails are cold sales email to generated leads and should catch the eye of the reader so they are inclined to open it up. \
                For context, my company is a space vacation business (which may or may not help with generating the subject). \
                Respond with just the subject line, nothing else. The user will provide the email body."
user_prompt = """
    Hi Jane,
    
    I heard you are interested in traveling to space for your family vacation next summer! At \
    Space Explorer's Inc. we can provide a comprehensive tour of the upper atmosphere of Earth at \
    a fraction of the cost of other tour companies! If you have the time, I'd love to chat to provide \
    you with more information.
    
    Best,
    Matt
"""

# Step 2: Make the messages list

messages = [{"role": "system", "content": system_prompt}, 
           {"role": "user", "content": user_prompt} ] # fill this in

# Step 3: Call OpenAI
response = openai.chat.completions.create(
    model = "gpt-4o-mini",
    messages = messages
)

# Step 4: print the result

print(response.choices[0].message.content)

"Unlock the Adventure of a Lifetime: Space Vacations for Your Family!"


## An extra exercise for those who enjoy web scraping

You may notice that if you try `display_summary("https://openai.com")` - it doesn't work! That's because OpenAI has a fancy website that uses Javascript. There are many ways around this that some of you might be familiar with. For example, Selenium is a hugely popular framework that runs a browser behind the scenes, renders the page, and allows you to query it. If you have experience with Selenium, Playwright or similar, then feel free to improve the Website class to use them. In the community-contributions folder, you'll find an example Selenium solution from a student (thank you!)

# Sharing your code

I'd love it if you share your code afterwards so I can share it with others! You'll notice that some students have already made changes (including a Selenium implementation) which you will find in the community-contributions folder. If you'd like add your changes to that folder, submit a Pull Request with your new versions in that folder and I'll merge your changes.

If you're not an expert with git (and I am not!) then GPT has given some nice instructions on how to submit a Pull Request. It's a bit of an involved process, but once you've done it once it's pretty clear. As a pro-tip: it's best if you clear the outputs of your Jupyter notebooks (Edit >> Clean outputs of all cells, and then Save) for clean notebooks.

Here are good instructions courtesy of an AI friend:  
https://chatgpt.com/share/677a9cb5-c64c-8012-99e0-e06e88afd293